## import libraries 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np

path = os.getcwd()
path

'/Users/rubenstuiver/code/rubenstuiver/olist/notebooks'

## Creating a variable csv_path that stores the path to data in the csv folder as a string

In [3]:
csv_path = '/Users/rubenstuiver/code/rubenstuiver/olist/raw_data/csv'
csv_path

'/Users/rubenstuiver/code/rubenstuiver/olist/raw_data/csv'

In [4]:
import pandas as pd
pd.read_csv(os.path.join(csv_path, 'olist_sellers_dataset.csv')).head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


## Checking all the different csv files in the folder

In [5]:
os.listdir(csv_path)

['olist_sellers_dataset.csv',
 'product_category_name_translation.csv',
 'olist_orders_dataset.csv',
 '.gitkeep',
 'olist_order_items_dataset.csv',
 'olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv']

In [6]:
len(os.listdir(csv_path))

10

In [7]:
file_names = [f for f in os.listdir(csv_path) if f.endswith('.csv')]
file_names

['olist_sellers_dataset.csv',
 'product_category_name_translation.csv',
 'olist_orders_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv']

In [8]:
len(file_names)

9

In [9]:
# file_names = str(file_names).replace("_dataset","")
# file_names = str(file_names).replace("olist_","")
# file_names = str(file_names).replace(".csv","")
# file_names

In [10]:
key_names = [key_name
    .replace('olist_','')
    .replace('_dataset','')
    .replace('.csv','')
    for key_name in file_names]
key_names

['sellers',
 'product_category_name_translation',
 'orders',
 'order_items',
 'customers',
 'geolocation',
 'order_payments',
 'order_reviews',
 'products']

In [11]:
## Creating the list of dict key key_names

In [12]:
data = {}
for key,file in zip(key_names, file_names):
    data[key] = pd.read_csv(os.path.join(csv_path, file))
data.keys()

dict_keys(['sellers', 'product_category_name_translation', 'orders', 'order_items', 'customers', 'geolocation', 'order_payments', 'order_reviews', 'products'])

##  Implement the method get_data() in olist/data.py


In [13]:
os.getcwd()

'/Users/rubenstuiver/code/rubenstuiver/olist/notebooks'

In [14]:
from olist.data import Olist
Olist().get_data()['sellers'].head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


## Exploratory Analysis

In [15]:
# import numpy as np
# import pandas as pd
# %load_ext autoreload
# %autoreload 2

In [16]:
# All data is still a dict
data = Olist().get_data()

All _id that are in the tables
- customer_id
- product_id
- seller_id
- review
- order_id

##  Run an automated exploratory analysis with pandas profiling

In [17]:
#install the pandas-profiling package
! pip install --quiet pandas-profiling

In [18]:
os.getcwd()

'/Users/rubenstuiver/code/rubenstuiver/olist/notebooks'

In [19]:
# And create a new "04-Decision-Science/reports" folder 
!mkdir -p ../raw_data/reports

In [20]:
import pandas_profiling
datasets_to_profile = ['orders', 'products', 'sellers',
                  'customers', 'order_reviews',
                  'order_items']

In [21]:
# for d in datasets_to_profile:
#     print('exporting: '+d)
#     profile = data[d].profile_report(title='Report for '+d)
#     profile.to_file(output_file="../raw_data/reports/"+d+'.html');

## Investigate the cardinalities of the various DataFrames (without using pandas profiling)

amount of unique: orders, reviews, sellers, products and customer

In [22]:
aggregates = np.array([
    [
        data["orders"].order_id.nunique(),
        data["order_reviews"].review_id.nunique(),
        data["sellers"].seller_id.nunique(),
        data["products"].product_id.nunique(),
        data["customers"].customer_id.nunique(),
    ],
    [
        data["orders"].order_id.count(),
        data["order_reviews"].review_id.count(),
        data["sellers"].seller_id.count(),
        data["products"].product_id.count(),
        data["customers"].customer_id.count(),
    ],
])

pd.DataFrame(data=np.vstack((aggregates, aggregates[0,:]/aggregates[1,:])).T,
             index=['orders', 'reviews', 'sellers', 'products', 'customers'],
            columns=['nunique', 'count','ratio'])

,nunique,count,ratio
orders,99441.0,99441.0,1.000000
reviews,98410.0,99224.0,0.991796
sellers,3095.0,3095.0,1.000000
products,32951.0,32951.0,1.000000
customers,99441.0,99441.0,1.000000


## amount of reviews per order

In [23]:
orders = data['orders'][['customer_id', 'order_id']]
reviews = data['order_reviews'][['order_id', 'review_id']]
items = data['order_items'][['order_id', 'product_id','seller_id']]

In [24]:
orders_reviews = orders.merge(reviews, on='order_id', how='outer')
orders_reviews.head(2)

,customer_id,order_id,review_id
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,a54f0611adc9ed256b57ede6b6eb5114
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,8d5266042046a06655c8db133d120ba5


In [25]:
# Check missing reviews
orders_reviews.isna().sum()

customer_id      0
order_id         0
review_id      768
dtype: int64

In [26]:
missing=orders_reviews.isna().sum()[2]

In [27]:
percent_missing_reviews_per_order=missing/len(reviews)*100
percent_missing_reviews_per_order

0.7740062888010965

In [28]:
orders_reviews.groupby(['order_id', 'customer_id']).count().hist()

array([[<AxesSubplot:title={'center':'review_id'}>]], dtype=object)

## First glimps of Olist (reviews/orders)

In [29]:
# Import Modules
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
# Loading the data
#from olist.data import Olist
data = Olist().get_data()

In [54]:
#Orders vs Reviews

In [31]:
data['order_reviews']['review_score'].unique()

array([4, 5, 1, 3, 2])

In [32]:
# histogram for the review score
sns.countplot(x='review_score', data=data['order_reviews'])

<AxesSubplot:title={'center':'review_id'}, xlabel='review_score', ylabel='count'>

In [33]:
# histogram of order status
plt.figure(figsize = (10,6))
sns.countplot(data = data['orders'], x = 'order_status')

<AxesSubplot:xlabel='order_status', ylabel='count'>

In [34]:
data['orders']['order_status'].value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

## Evolution of key business metrics over the past 14 months

### Metric 1: Order numbers



In [35]:
# Orders dataset
orders = data['orders'].copy()
# Handle datetime
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders = orders.set_index('order_purchase_timestamp').sort_index()
orders

,order_id,customer_id,order_status,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
order_purchase_timestamp,,,,,,,
2016-09-04 21:15:19,2e7a8482f6fb09756ca50c10d7bfc047,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-10-07 13:18:03,2016-10-18 13:14:51,NaN,2016-10-20 00:00:00
2016-09-05 00:15:34,e5fa5a7210941f7d56d0208e4e071d35,683c54fc24d40ee9f8a6fc179fd9856c,canceled,2016-10-07 13:17:15,NaN,NaN,2016-10-28 00:00:00
2016-09-13 15:24:19,809a282bbd5dbcabb6f2f724fca862ec,622e13439d6b5a0b486c435618b2679e,canceled,2016-10-07 13:16:46,NaN,NaN,2016-09-30 00:00:00
2016-09-15 12:16:38,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00
2016-10-02 22:07:52,71303d7e93b399f5bcd537d124c0bcfa,b106b360fe2ef8849fbbd056f777b4d5,canceled,2016-10-06 15:50:56,NaN,NaN,2016-10-25 00:00:00
...,...,...,...,...,...,...,...
2018-09-29 09:13:03,392ed9afd714e3c74767d0c4d3e3f477,2823ffda607a2316375088e0d00005ec,canceled,NaN,NaN,NaN,2018-10-15 00:00:00
2018-10-01 15:30:09,616fa7d4871b87832197b2a137a115d2,bf6181a85bbb4115736c0a8db1a53be3,canceled,NaN,NaN,NaN,2018-10-23 00:00:00
2018-10-03 18:55:29,a2ac6dad85cf8af5b0afb510a240fe8c,4c2ec60c29d10c34bd49cb88aa85cfc4,canceled,NaN,NaN,NaN,2018-10-16 00:00:00


In [36]:
orders.resample('W')['order_id'].count().plot(color='g')
plt.title('weekly orders (total)')

Text(0.5, 1.0, 'weekly orders (total)')

In [37]:
orders.sort_index().query("order_status != 'delivered'").loc['2017-02-01':].resample('W')['order_id'].count().plot(color='r')
plt.title('weekly non-delivered orders')

Text(0.5, 1.0, 'weekly non-delivered orders')

### Metric 2: Customer Satisfaction

In [38]:
reviews = data['order_reviews'].copy()

In [39]:
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [40]:
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
reviews.set_index('review_creation_date', inplace=True)

In [41]:
reviews

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_answer_timestamp
review_creation_date,,,,,,
2018-01-18,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 21:46:59
2018-03-10,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-11 03:05:13
2018-02-17,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-18 14:36:24
2017-04-21,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 22:02:06
2018-03-01,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-02 10:26:53
...,...,...,...,...,...,...
2018-07-07,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,NaN,NaN,2018-07-14 17:18:30
2017-12-09,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-11 20:06:42
2018-03-22,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-23 09:10:43


In [42]:
# customer satisfaction evolution over time
reviews.loc['2017-02-01':].resample('M').agg({'review_score':'mean'}).plot()

<AxesSubplot:xlabel='review_creation_date'>

### Metric 3: Shipping delay

In [43]:
# keep only delivered items
orders = orders.query("order_status=='delivered'").reset_index()

In [44]:
# datetime 
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

In [45]:
orders['delay_vs_expected'] = (orders['order_delivered_customer_date'] - orders['order_estimated_delivery_date']) / np.timedelta64(24, 'h')

In [46]:
orders['wait_time'] = (orders['order_delivered_customer_date'] -\
                       orders['order_purchase_timestamp']) / np.timedelta64(24, 'h')
orders['wait_time']

0        54.813194
1        23.178738
2        24.057500
3        30.572581
4        27.542813
           ...    
96473     1.432269
96474     1.087870
96475     1.106979
96476     1.072905
96477     1.057627
Name: wait_time, Length: 96478, dtype: float64

In [47]:
# a boolean feature that says whether an order was delivered with a delay or not
# delay_vs_expected > 0 
# <=> actual_delivered date > ETA 
# <=> actual_delivered date - ETA > 0

orders["was_late"] = orders['delay_vs_expected'].map(lambda x: x > 0)

In [48]:
orders['was_late'].value_counts()

False    88652
True      7826
Name: was_late, dtype: int64

In [49]:
round(orders['was_late'].value_counts(normalize=True),2)

False    0.92
True     0.08
Name: was_late, dtype: float64

* On average, 8% of the orders are delivered after the ETA

In [50]:
# find the worst week

In [51]:
delay_analysis_per_week = orders.set_index('order_purchase_timestamp').resample('W').agg({
    'delay_vs_expected':np.mean,
    'wait_time':np.mean,
    'was_late':np.sum,
    'order_id':'count'})

delay_analysis_per_week.columns = ['avg_delay_vs_exp', 'avg_wait_time','nb_of_delays', 'nb_of_orders']

delay_analysis_per_week['pct_of_lateness'] = delay_analysis_per_week['nb_of_delays'] / delay_analysis_per_week['nb_of_orders']

print("-"*40)
print("Delay Analysis per week")
print("-"*40)
display(round(delay_analysis_per_week.tail(10),2))

----------------------------------------
Delay Analysis per week
----------------------------------------


,avg_delay_vs_exp,avg_wait_time,nb_of_delays,nb_of_orders,pct_of_lateness
order_purchase_timestamp,,,,,
2018-07-01,-16.78,8.33,10,1382,0.01
2018-07-08,-13.84,8.52,19,1178,0.02
2018-07-15,-9.88,11.27,59,972,0.06
2018-07-22,-10.40,9.03,65,1649,0.04
2018-07-29,-9.92,8.32,63,1610,0.04
2018-08-05,-6.57,8.66,276,2002,0.14
2018-08-12,-7.01,8.49,243,1931,0.13
2018-08-19,-8.52,7.63,133,1839,0.07
2018-08-26,-8.32,5.48,71,1054,0.07


In [52]:
#number of delays and percentage of delayed orders per week
with plt.style.context('seaborn-deep'):
    # figsize
    plt.figure(figsize=(14,8))
    # getting axes
    ax1 = plt.gca()
    # duplicating the ax1
    ax2 = ax1.twinx()
    # Weekly payments and Mean Payments
    
    ax1.plot(delay_analysis_per_week['nb_of_orders'],
             color="blue",
             label="Number of orders")
    ax1.set_xlabel("Weeks", 
                   fontsize = 16, 
                   labelpad = 10)
    ax1.set_ylabel("Late orders per week", 
                   fontsize = 16, 
                   labelpad = 20)

    # Numbers
    ax2.plot(delay_analysis_per_week['pct_of_lateness'],
             label="Percentage of lateness",
             c="purple")
    ax2.set_ylabel("Percentage of lateness", 
                   fontsize = 16, 
                   rotation = 90*3, 
                   labelpad = 20)

    # Get a conjoined legend
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper left', fontsize = 14)

    # Title
    ax1.set_title('Late deliveries per week', 
                  fontsize = 20, 
                  pad = 10)
    

    ax1.grid(axis="x",linewidth=0.5)
    plt.show()

/var/folders/jv/yk7cpcx97ln8wgt109nxk6c00000gn/T/ipykernel_83419/812261706.py:42: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
